In [201]:
import numpy as np
import torch
from torch import nn

import matplotlib.pyplot as plt

In [202]:
# 构造图像数据

sample_num, height, width, channel, n_classes = 100, 28, 28, 1, 10

x = np.random.randint(low=0, high=255+1, size=(sample_num, channel, width, height))
y = np.random.randint(low=0, high=n_classes, size=(sample_num, ))

x = x.astype(np.float32)
y = y.astype(np.float32)

print(f"x shape {x.shape}, y shape {y.shape}")

x shape (100, 1, 28, 28), y shape (100,)


In [203]:
# 定义 Convolution Network 模型

class CNN(nn.Module):
    def __init__(self, input_dims, n_classes):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=input_dims, out_channels=6, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 10, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )

        self.fc = nn.Sequential(
            nn.Linear(250, 120),
            nn.Linear(120, 84),
            nn.Linear(84, n_classes)
        )

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

In [204]:
model = CNN(input_dims=channel, n_classes=n_classes)  # 图片大小是28x28

use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()

# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [205]:
# 开始训练
x_train, y_train = torch.from_numpy(x), torch.LongTensor(y)
# torch.zeros(sample_num, n_classes).scatter_(1, torch.LongTensor(y), 1)
# y_train = type(torch.LongTensor)

num_epochs = 500
for epoch in range(num_epochs):
    # forward
    out = model(x_train)
    loss = criterion(out, y_train)
    # backward
    optimizer.zero_grad()  # 梯度归零
    loss.backward()  # 梯度反向传播
    optimizer.step()  # 参数更新

    if (epoch+1) % 20 == 0:
        print(f'Epoch[{epoch+1}/{num_epochs}], loss: {loss.item():.6f}')

Epoch[20/500], loss: 2.217970
Epoch[40/500], loss: 1.775037
Epoch[60/500], loss: 1.405125
Epoch[80/500], loss: 1.100280
Epoch[100/500], loss: 0.847788
Epoch[120/500], loss: 0.656040
Epoch[140/500], loss: 0.491634
Epoch[160/500], loss: 0.370614
Epoch[180/500], loss: 0.284792
Epoch[200/500], loss: 0.222978
Epoch[220/500], loss: 0.178232
Epoch[240/500], loss: 0.145553
Epoch[260/500], loss: 0.121396
Epoch[280/500], loss: 0.103147
Epoch[300/500], loss: 0.088907
Epoch[320/500], loss: 0.077538
Epoch[340/500], loss: 0.068367
Epoch[360/500], loss: 0.060847
Epoch[380/500], loss: 0.054632
Epoch[400/500], loss: 0.049418
Epoch[420/500], loss: 0.044993
Epoch[440/500], loss: 0.041214
Epoch[460/500], loss: 0.037942
Epoch[480/500], loss: 0.035090
Epoch[500/500], loss: 0.032593


In [206]:
with torch.no_grad():
    predict = model(x_train)
predict = predict.data.numpy()

In [207]:
predict

array([[-3.13575125e+00, -1.72840130e+00,  4.67784500e+00,
        -4.01397324e+00, -3.89514661e+00,  7.61057317e-01,
        -1.54239106e+00,  2.11414665e-01, -2.76914763e+00,
        -2.83242583e+00],
       [ 3.91973114e+00, -2.43993878e+00, -1.58696675e+00,
        -2.89141035e+00, -5.07920563e-01, -3.80830216e+00,
         1.59064993e-01, -2.11978173e+00, -3.35860682e+00,
        -5.44686604e+00],
       [-4.46535110e+00, -4.26043844e+00, -9.60532427e-01,
         3.97506857e+00, -2.48749924e+00, -4.88375336e-01,
        -6.40912294e-01, -3.70692462e-01, -4.07623339e+00,
        -1.17449534e+00],
       [-4.01104069e+00, -2.33765650e+00, -1.44171143e+00,
         4.05669403e+00, -3.60355520e+00, -2.36450434e+00,
        -1.75471139e+00, -2.34603405e+00, -5.52290535e+00,
        -2.35420287e-01],
       [-1.50763440e+00, -9.79402736e-02,  2.32656389e-01,
        -2.28321981e+00, -2.88564968e+00, -6.84782565e-01,
        -1.88318527e+00, -1.31003308e+00, -5.91762829e+00,
         4.

In [155]:
# 损失调试

# import torch
# import torch.nn as nn

# model = nn.Linear(10, 3)
# criterion = nn.CrossEntropyLoss()

# x = torch.randn(16, 10)
# y = torch.randint(0, 3, size=(16,))  # (16, )
# logits = model(x)  # (16, 3)

# loss = criterion(logits, y)